In [ ]:
import wptherml
from matplotlib import pyplot as plt
import numpy as np
import os
from scipy import interpolate
import scipy
import glob
import colour
from colour.plotting import *

# create a tungsten incandescent object at 1700 K
tungsten_values = {
    "wavelength_list": [400e-9, 700e-9, 200],
    "Material_List": ["Air",  "W", "Air"],
    "Thickness_List": [0, 900e-9, 0],
    "temperature": 1700,
    "therml": True
}


sf = wptherml.SpectrumFactory()
w = sf.spectrum_factory('Tmm', tungsten_values)


Read in the emission spectra data from the Meth_and_Propoxy_TTz folder!

In [ ]:
file_location = os.path.join('Meth_and_Butoxy_TTz_sub','*.txt')
filenames = glob.glob(file_location)
print(filenames)

Now read in the emission spectra from each file, compute the CIE and RGB values for each spectrum, print
the filename and CIE values, and store the RGB values to a list.  We will use that RGB list subsequently to plot the values on the CIE color gamut.

In [ ]:
cie_x = []
cie_y = []
rgb_list = []
for file in filenames:
    data = np.loadtxt(file)
    emission_spline = interpolate.interp1d(data[:,0], data[:,1])
    # make sure this spline is good!
    assert np.allclose(emission_spline(data[:,0]), data[:,1], 1e-4)
    emission_prediction = emission_spline(w.wavelength_array * 1e9)   
    cie = w.compute_cie(emission_prediction)
    cie_x.append(cie[0])
    cie_y.append(cie[1])
    rgb = w.compute_rgb(emission_prediction)
    rgb_list.append(rgb)
    print(F"{file[20:]}, {cie[0]}, {cie[1]}, {cie[2]}")

Notes to Lane: 
1. I found some syntax that allows plotting multiple layers on the CIE diagram, which allows us to plot the CIE values directly (i.e. no more mapping them to RGB values), and will also allow us to plot a line of best fit.

2. Right now, I am plotting both a scatter plot and a line through all the points (rather than a line of best fit) with the following code:

`plt.scatter(cie_x, cie_y)
plt.plot(cie_x, cie_y, 'black')`

You can find a line of best fit using the scipy.stats function `linregress` as follows:

`slope, intercept, r, p, se = scipy.stats.linregress(cie_x, cie_y)`

and then define a line of best fit along the actual CIE x values simply as 

`y_fit = slope * cie_x`

You can then change the one `plt.plot()` command to 
`plt.plot(cie_x, cie_y, 'black')`
and hopefully that will plot the line of best fit on top of the actual CIE values.



In [ ]:
# ==> Add line to compute slope, etc, using scipy.stats.linregress here! <==

In [ ]:
colour.plotting.plot_chromaticity_diagram_CIE1931(standalone=False)
plt.scatter(cie_x, cie_y)
# ==> Change the plt.plot() line below so that it is plotting the y-vales associated
# ==> with the slope * cie_x where slope comes from scipy.stats.linregress
plt.plot(cie_x, cie_y, 'black') 
plt.title("")
colour.plotting.render(
    standalone=True,
    bounding_box = (-0.1, 0.9, -0.1, 0.9)
)